# Práctica 4.a

In [2]:
from ultralytics import YOLO
import numpy as np
import cv2  
import math 
import os
from collections import defaultdict
from roboflow import Roboflow

## Deteción de personas y vehículos presentes

In [45]:
#Definimos el modelo para separar los objetos que son personas y coches
model = YOLO("yolo11n.pt")
classes = {0:"person",2:"car"}

capture_video = cv2.VideoCapture("video1.mp4")
while(True):
    ret,frame_video = capture_video.read()
    if  ret:
        results = model(frame_video,stream=True)
        #Detección empezamos
        for frames in results:
            boxes = frames.boxes
            for box in boxes:
                # Clase
                cls = int(box.cls[0])
                if cls not in classes.keys():
                    continue
                else:
                    print("Clase -->",classes[cls])
                    x1, y1, x2, y2 = box.xyxy[0]
                    x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2) # convert to int values
                        
                    # Confianza
                    confidence = math.ceil((box.conf[0]*100))/100
                    print("Confianza --->",confidence)
                    escala = int((cls / len(classes)) * 255 * 3)
                    if escala >= 255*2:
                        R = 255
                        G = 255
                        B = escala - 255*2
                    else:
                        if escala >= 255:
                            R = 255
                            G = escala - 255
                            B = 0
                        else:
                            R = escala
                            G = 0
                            B = 0

                        # Dibuja el contenedor y clase
                    cv2.rectangle(frame_video, (x1, y1), (x2, y2), (R, G, B), 3)
                    cv2.putText(frame_video, classes[cls] , [x1, y1], cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, B), 2)
            cv2.imshow('capture_video', frame_video)
        if cv2.waitKey(20) == 27:
            break
# Libera el objeto de captura
capture_video.release()
# Destruye ventanas
cv2.destroyAllWindows()
            


0: 384x640 5 persons, 2 bicycles, 3 cars, 4 traffic lights, 36.7ms
Clase --> car
Confianza ---> 0.93
Clase --> car
Confianza ---> 0.81
Clase --> person
Confianza ---> 0.7
Clase --> car
Confianza ---> 0.58
Clase --> person
Confianza ---> 0.38
Clase --> person
Confianza ---> 0.36
Clase --> person
Confianza ---> 0.33
Clase --> person
Confianza ---> 0.26
Speed: 56.1ms preprocess, 36.7ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 3 bicycles, 3 cars, 4 traffic lights, 43.5ms
Clase --> car
Confianza ---> 0.93
Clase --> car
Confianza ---> 0.81
Clase --> person
Confianza ---> 0.7
Clase --> car
Confianza ---> 0.59
Clase --> person
Confianza ---> 0.38
Clase --> person
Confianza ---> 0.37
Clase --> person
Confianza ---> 0.33
Clase --> person
Confianza ---> 0.27
Speed: 6.4ms preprocess, 43.5ms inference, 5.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 persons, 3 bicycles, 3 cars, 4 traffic lights, 29.1ms
Clase --> car
Confianza ---> 

# Deteción de matrículas presentes

In [ ]:

rf = Roboflow(api_key="")
project = rf.workspace("licenseplate-s6fjf").project("license-plate-xmnzu")
version = project.version(1)
dataset = version.download("yolov11")
#Funciona
                

loading Roboflow workspace...
loading Roboflow project...


In [ ]:
model = YOLO("yolo11n.pt")
data_path = "./license-plate-1/data.yaml"
results = model.train(data=data_path,epochs=15,imgsz=640)

In [ ]:
#Definimos el modelo para separar los objetos que son personas y coches
model = YOLO("./runs/detect/train/weights/best.pt")
classes = {0:"licenseplate",1:"car"}

capture_video = cv2.VideoCapture("Video.mp4")
while(True):
    ret,frame_video = capture_video.read()
    if  ret:
        results = model(frame_video,stream=True)
        #Detección empezamos
        for frames in results:
            boxes = frames.boxes
            for box in boxes:
                # Clase
                cls = int(box.cls[0])
                if cls not in classes.keys():
                    continue
                else:
                    print("Clase -->",classes[cls])
                    x1, y1, x2, y2 = box.xyxy[0]
                    x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2) # convert to int values
                        
                    # Confianza
                    confidence = math.ceil((box.conf[0]*100))/100
                    print("Confianza --->",confidence)
                    escala = int((cls / len(classes)) * 255 * 3)
                    if escala >= 255*2:
                        R = 255
                        G = 255
                        B = escala - 255*2
                    else:
                        if escala >= 255:
                            R = 255
                            G = escala - 255
                            B = 0
                        else:
                            R = escala
                            G = 0
                            B = 0

                        # Dibuja el contenedor y clase
                    cv2.rectangle(frame_video, (x1, y1), (x2, y2), (R, G, B), 3)
                    cv2.putText(frame_video, classes[cls] , [x1, y1], cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, B), 2)
            cv2.imshow('capture_video', frame_video)
        if cv2.waitKey(20) == 27:
            break
# Libera el objeto de captura
capture_video.release()
# Destruye ventanas
cv2.destroyAllWindows()

# Seguimiento de personas y coches

In [ ]:
import cv2
from ultralytics import YOLO
import math
from collections import defaultdict
# Necesitas numpy para la parte de polylines, asegúrate de tenerlo instalado: pip install numpy
import numpy as np 

# 1. Carga del modelo (yolo11n.pt, genérico para detectar personas y coches)
model = YOLO('yolo11n.pt') 

# 2. Etiqueta de las distintas clases (quitamos 'bicycle')
# COCO IDs: 0="person", 2="car"
classNames = {0: "person", 2: "car"}

# IDs de clases que queremos detectar
CLASSES_TO_TRACK = [0, 2] 

# Captura desde el video
vid = cv2.VideoCapture("video1.mp4")

# Estructuras para el Conteo Único
contador_clases_unicas = {nombre: 0 for nombre in classNames.values()}
tracker_ids_contados = set()
COLOR_TEXTO_CONTEO = (0, 255, 255) # Amarillo brillante BGR

# Para la trayectoria (manteniendo tu lógica)
track_history = defaultdict(lambda: [])

while(True): 
    ret, img = vid.read()
    if ret: 
        results = model.track(img, persist=True, classes=CLASSES_TO_TRACK, tracker="bytetrack.yaml", verbose=False)
        if not results or results[0].boxes.id is None:
            pass
        else:
           
            boxes = results[0].boxes.xywh.cpu()
            track_ids = results[0].boxes.id.int().cpu().tolist()
            for box, track_id in zip(boxes, track_ids):
                x1, y1, w, h = box.tolist() 
                x1, y1, x2, y2 = int(x1 - w/2), int(y1 - h/2), int(x1 + w/2), int(y1 + h/2)
                cls_tensor = results[0].boxes.cls.cpu().numpy()
                conf_tensor = results[0].boxes.conf.cpu().numpy()
                try:
                    box_index = results[0].boxes.id.int().cpu().tolist().index(track_id)
                    cls = int(cls_tensor[box_index])
                    confidence = math.ceil((conf_tensor[box_index]*100))/100
                    clase_nombre = classNames[cls]
                except ValueError:
                    continue
                if track_id not in tracker_ids_contados:
                    contador_clases_unicas[clase_nombre] += 1
                    tracker_ids_contados.add(track_id)
                    
                    
                escala = int((cls / len(CLASSES_TO_TRACK)) * 255 * 3) 
                R, G, B = 0, 0, 0
                
                if escala >= 255*2:
                    R, G, B = 255, 255, escala - 255*2
                elif escala >= 255:
                    R, G, B = 255, escala - 255, 0
                else:
                    R, G, B = escala, 0, 0
                    
                color = (R, G, B) # BGR
                
                cv2.rectangle(img, (x1, y1), (x2, y2), color, 3)
                
                etiqueta = f"ID:{track_id} {clase_nombre} ({confidence:.2f})"
                cv2.putText(img, etiqueta, [x1, y1 - 10], cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

                track = track_history[track_id]
                track.append((float(x1 + w/2), float(y1 + h/2))) 
                if len(track) > 30:
                    track.pop(0)
                points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
                cv2.polylines(img, [points], isClosed=False, color=(230, 230, 230), thickness=10)

        y_offset = 30
        cv2.putText(img, "--- CONTEO UNICO ACUMULADO ---", (10, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2, cv2.LINE_AA)
        y_offset += 30
        
        for nombre, cantidad in contador_clases_unicas.items():
            texto_conteo = f"Total Unicos {nombre}: {cantidad}"
            cv2.putText(img, texto_conteo, (10, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.8, COLOR_TEXTO_CONTEO, 2, cv2.LINE_AA) 
            y_offset += 30


        cv2.imshow('YOLO Tracking & Unique Count', img)
    
    if cv2.waitKey(20) == 27:
        break
    
    if not ret:
        break

vid.release()
cv2.destroyAllWindows()

print("\n--- RESUMEN FINAL DE CONTEO DE OBJETOS ÚNICOS ---")
for nombre, cantidad in contador_clases_unicas.items():
    print(f"Total acumulado de objetos únicos ({nombre}) detectados: {cantidad}")

# Seguimiento de matriculas y coches

In [ ]:
import cv2
from ultralytics import YOLO
import math
from collections import defaultdict
import numpy as np 
import os 

MODELO_PATH = "./runs/detect/train/weights/best.pt"

model = YOLO(MODELO_PATH) 



classes = {0: "Matricula", 1: "Coche"}
CLASSES_TO_TRACK = list(classes.keys()) 
VID_PATH = "Video.mp4"
capture_video = cv2.VideoCapture(VID_PATH)
contador_clases_unicas = {nombre: 0 for nombre in classes.values()}
tracker_ids_contados = set()


COLOR_TEXTO_CONTEO = (0, 255, 255) 
track_history = defaultdict(lambda: []) 


while(True): 
    ret, img = capture_video.read()

    if ret: 
        results = model.track(
            img, 
            persist=True, 
            classes=CLASSES_TO_TRACK, 
            tracker="bytetrack.yaml", 
            verbose=False
        )

    
        if not results or results[0].boxes.id is None:
            pass 
        else:
            boxes_data = results[0].boxes 
            
            for i, box in enumerate(boxes_data):
                cls = int(box.cls[0])
                conf = box.conf[0].item()
                track_id = int(box.id[0].item()) if box.id is not None else None
                
                x1, y1, x2, y2 = [int(val) for val in box.xyxy[0].tolist()]
                
                if cls in classes.keys():
                    clase_nombre = classes[cls]
                    
                
                    if track_id is not None and track_id not in tracker_ids_contados:
                        contador_clases_unicas[clase_nombre] += 1
                        tracker_ids_contados.add(track_id)
                      
                    escala = int((cls / len(classes)) * 255 * 3)
                    R, G, B = 0, 0, 0
                    if escala >= 255*2:
                        R, G, B = 255, 255, escala - 255*2
                    elif escala >= 255:
                        R, G, B = 255, escala - 255, 0
                    else:
                        R, G, B = escala, 0, 0
                    color = (R, G, B) 
                    
                    
                    cv2.rectangle(img, (x1, y1), (x2, y2), color, 3)
                    
                    
                    etiqueta = f"ID:{track_id} {clase_nombre} ({conf*100:.0f}%)"
                    cv2.putText(img, etiqueta, [x1, y1 - 10], cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
                    
                    
                    center_x, center_y = (x1 + x2) // 2, (y1 + y2) // 2
                    track = track_history[track_id]
                    track.append((float(center_x), float(center_y)))
                    if len(track) > 30: 
                        track.pop(0)
                    points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
                    cv2.polylines(img, [points], isClosed=False, color=(230, 230, 230), thickness=5)


    
        y_offset = 30
        cv2.putText(img, "--- CONTEO UNICO ACUMULADO ---", (10, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2, cv2.LINE_AA)
        y_offset += 30
        
        for nombre, cantidad in contador_clases_unicas.items():
            texto_conteo = f"Total Unicos {nombre}: {cantidad}"
            cv2.putText(img, texto_conteo, (10, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.8, COLOR_TEXTO_CONTEO, 2, cv2.LINE_AA) 
            y_offset += 30

        cv2.imshow('Deteccion y Conteo Unico de Trafico', img)
    

    if cv2.waitKey(20) == 27:
        break   
    
    if not ret:
        break
capture_video.release()
cv2.destroyAllWindows()

print("\n--- RESUMEN FINAL DE CONTEO DE OBJETOS ÚNICOS ---")
for nombre, cantidad in contador_clases_unicas.items():
    print(f"Total acumulado de objetos únicos ({nombre}) detectados: {cantidad}")

## 4B 